
# train_deepfakedetection
### Import Python Packages


In [1]:
from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils import Sequence, to_categorical
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model

print("yolo")
import os

import glob
import re

import numpy as np
import shutil
from CLRNet_convlstm import CLRNet
from Utility_functions import create_sequence
from DFVDSequence import DFVDSequence
from PIL import Image
import random
from sklearn.utils import class_weight
from datetime import datetime as dt
import natsort
import matplotlib.pyplot as plt
import cv2
random.seed(32)
dataset_dir='DeepFakeDatasetReal'
import ipykernel
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import backend as K

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# Choose GPU NUMBERS [0, 1, 2, 3]
os.environ["CUDA_VISIBLE_DEVICES"]="1"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

# Data Loaders

In [2]:
train_video_per_batch=16
val_video_per_batch=16
test_video_per_batch=16
frames_per_video=80 #total frames
frames_per_video_per_batch=5 #frames in one batch
image_size=240

channel=3
X_train,y_train, class_weights_train =create_sequence('datasets/DeepFakeDetection/train/',frames_per_video_per_batch,frames_per_video)
X_val,y_val,class_weights_val=create_sequence('datasets/DeepFakeDetection/val/',frames_per_video_per_batch,frames_per_video)
# X_test,y_test,class_weights_test=create_sequence('datasets/Deepfakes/test/',frames_per_video_per_batch,frames_per_video)
train_it=DFVDSequence(X_train,y_train,train_video_per_batch,frames_per_video_per_batch,image_size,True)
val_it=DFVDSequence(X_val,y_val,val_video_per_batch,frames_per_video_per_batch,image_size,False)
# test_it=DFVDSequence(X_test,y_test,test_video_per_batch,frames_per_video_per_batch,image_size)

fig=plt.figure(figsize=(10, 100))
columns = 4
rows = 23

print("Training")
x=1
temp=train_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
plt.imshow(np.array(temp[0][0,0]))
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][1,0]))
print(temp[0].shape,temp[1].shape,train_it.dataset_size,train_it.frame_counter,train_it.__len__(),temp[1][0],temp[1][1])
# x+=2
plt.title("Training Set Examples")        
plt.show()
train_it.on_epoch_end()
fig=plt.figure(figsize=(10, 100))
columns = 4
rows = 23

print("Validation")
x=1
temp=val_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
plt.imshow(np.array(temp[0][0,0]),)
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][1,0]))
print(temp[0].shape,temp[1].shape,val_it.dataset_size,val_it.frame_counter,val_it.__len__(),temp[1][0],temp[1][1])
plt.title("Validation Set Examples")   
plt.show()
val_it.on_epoch_end()
                                                                                                            

# Model Compile

In [3]:
model=CLRNet(input_shape=(frames_per_video_per_batch,image_size, image_size, channel), classes=2, block='bottleneck', residual_unit='v2',
           repetitions=[3, 4, 6, 3], initial_filters=64, activation='softmax', include_top=False,
           input_tensor=None, dropout=0.25, transition_dilation_rate=(1, 1),
           initial_strides=(2, 2), initial_kernel_size=(7, 7), initial_pooling='max',
           final_pooling=None, top='classification')
print_summary(model, line_length=150, positions=None, print_fn=None)


In [4]:
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# Choose GPU NUMBERS [0, 1, 2, 3]
# os.environ["CUDA_VISIBLE_DEVICES"]="2"
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)
# K.set_session(sess)
# _id = dt.now().strftime("%y-%m-%d-%H_%M")
# os.path.join('models','df_CLRNet50(bk)_',_id)
_id = dt.now().strftime("%y-%m-%d-%H_%M")
save_dir=os.path.join(os.getcwd(),'models','DeepFakeDetection','CLRNet',_id)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


filepath=os.path.join(save_dir,"{epoch:02d}-{acc:.2f}"+".hdf5")
checkpoint = ModelCheckpoint(filepath)#, monitor='acc', verbose=2, save_best_only=True, mode='max')
csv_logger = CSVLogger(os.path.join(save_dir,"training.csv"), append=True, separator=',')

# filepath=os.path.join('models','df_CLRNet50(bk)_'+_id+"-{epoch:02d}-{acc:.2f}"
# checkpoint = ModelCheckpoint(str(filepath)+".hdf5")#, monitor='acc', verbose=2, save_best_only=True, mode='max')
# csv_logger = CSVLogger('df_CLRNet50(bk)_'+str(_id)+".csv", append=True, separator=',')
# earlyStop=EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
callbacks_list = [checkpoint,csv_logger]


# Training

In [ ]:
model.fit_generator(generator=train_it, validation_data=val_it,epochs=300,
                    callbacks=callbacks_list,shuffle=False,class_weight=class_weights_train,verbose=1)

## load model from file

In [ ]:
model = load_model('models/NeuralTexture/CLRNet50(bk)/20-01-06-03_20/05-0.98.hdf5')
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])


In [ ]:
model.fit_generator(generator=train_it, validation_data=val_it,epochs=300,initial_epoch=5,
                    callbacks=callbacks_list,shuffle=False,class_weight=class_weights_train,verbose=1)


# test Model


In [ ]:

X_test,y_test,class_weights_test=create_sequence('datasets/Deepfakes/test/',frames_per_video_per_batch,frames_per_video)
test_it=DFVDSequence(X_test,y_test,test_video_per_batch,frames_per_video_per_batch)
# for i in range(0,test_it.__len__()):
#     temp=test_it.__getitem__(1)
#     print(temp[0].shape,temp[1].shape,test_it.dataset_size,test_it.frame_counter)
# # model.load_weights('2019-11-01_16.57.45_weights-improvement-01-1.00.hdf5')
# model.compile(loss='binary_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])
print(model.evaluate_generator(test_it))
test_it=DFVDSequence(X_test,y_test,test_video_per_batch,frames_per_video_per_batch)
y_prob=model.predict_generator(test_it)
y_classes = y_prob.argmax(axis=-1)
print(y_classes)
y_prob=model.predict_generator(test_it)
y_classes = y_prob.argmax(axis=-1)
print(y_classes)
# print('test loss:',loss)
# model=cl_basic(128,128,3,2)
# print_summary(model, line_length=150, positions=None, print_fn=None)
# model.compile(loss='binary_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])

In [ ]:
test_it=DFVDSequence(X_test,y_test,test_video_per_batch,frames_per_video_per_batch)
y_prob = model.evaluate_generator(test_it)
print(y_prob)
# y_classes = y_prob.argmax(axis=-1)
# print(y_classes)

In [ ]:

# model.fit_generator(generator=train_it, validation_data=val_it,epochs=150,initial_epoch=50,
                    callbacks=callbacks_list,shuffle=False,class_weight=class_weights_train,verbose=1)

In [ ]:
# _id = dt.now().strftime("%y-%m-%d-%H_%M")
# print(_id)

In [ ]:
# model.fit_generator(generator=train_it, validation_data=val_it,epochs=300,initial_epoch=150,
#                     callbacks=callbacks_list,shuffle=False,class_weight=class_weights_train,verbose=1)

In [ ]:
_id = dt.now().strftime("%y-%m-%d-%H_%M")
save_dir=os.path.join(os.getcwd(),'models','Deepfakes','CLRNet50(bk)',_id)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
os.getcwd()